<a href="https://colab.research.google.com/github/RainaDestiani/Tugas-Sesi-9-Latihan-Praktek/blob/main/Tugas3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers datasets scikit-learn evaluate


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset, DatasetDict
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import os

In [ ]:
import transformers
print(f"Transformers version: {transformers.__version__}")

Transformers version: 4.51.3


In [ ]:
cache_dir = "./model_cache"
os.makedirs(cache_dir, exist_ok=True)
print(f"Model akan disimpan di: {cache_dir}")

Model akan disimpan di: ./model_cache


In [ ]:
file_path = "/content/Review Instagram.csv"

In [ ]:
df = pd.read_csv(file_path)

In [ ]:
def map_sentiment(score):
    if score <= 2:
        return "negative"
    elif score == 3:
        return "neutral"
    else:
        return "positive"

In [ ]:
df = df[["Review Text", "Rating"]].dropna()
df["label"] = df["Rating"].apply(map_sentiment)


In [ ]:
le = LabelEncoder()
df["label_id"] = le.fit_transform(df["label"])


In [ ]:
print(f"Label distribution: {df['label'].value_counts()}")
print(f"Label mapping: {dict(zip(le.classes_, le.transform(le.classes_)))}")

Label distribution: label
negative    637
positive    229
neutral     134
Name: count, dtype: int64
Label mapping: {'negative': np.int64(0), 'neutral': np.int64(1), 'positive': np.int64(2)}


In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["Review Text"].tolist(),
    df["label_id"].tolist(),
    test_size=0.2,
    stratify=df["label_id"],
    random_state=42
)
print(f"Data training: {len(train_texts)}, Data validasi: {len(val_texts)}")

Data training: 800, Data validasi: 200


In [ ]:
print("Memuat model IndoBERT - ini mungkin akan mengunduh model jika belum ada di cache...")
model_name = "indolem/indobert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir, local_files_only=False)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
    cache_dir=cache_dir,
    local_files_only=False
)
print("Model berhasil dimuat!")


Memuat model IndoBERT - ini mungkin akan mengunduh model jika belum ada di cache...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model berhasil dimuat!


In [ ]:
dataset = DatasetDict({
    'train': Dataset.from_dict({"text": train_texts, "label": train_labels}),
    'validation': Dataset.from_dict({"text": val_texts, "label": val_labels})
})

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding=True)

print("Melakukan tokenisasi dataset...")
tokenized_dataset = dataset.map(tokenize, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
print("Tokenisasi selesai!")

Melakukan tokenisasi dataset...


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Tokenisasi selesai!


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    learning_rate=2e-5,
    logging_dir="./logs",
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    do_eval=True,
    report_to=["none"],        # <<< tambahkan ini
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

<ipython-input-69-ea83d2950a86>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train
print("Mulai fine-tuning...")
trainer.train()
print("Fine-tuning selesai!")


Mulai fine-tuning...


Step,Training Loss
100,0.834000


Step,Training Loss
100,0.834000


Fine-tuning selesai!


In [ ]:
model_save_path = "./finetuned-indobert-instagram"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"Model disimpan di: {model_save_path}")

Model disimpan di: ./finetuned-indobert-instagram


In [ ]:
sample_texts = [
    "Akun saya tidak bisa login, sangat mengecewakan",
    "Update terbaru sangat membantu, makin keren",
    "Biasa saja, belum terlalu terasa bedanya"
]

In [ ]:
print("\nPrediksi Sampel:")
encoded = tokenizer(sample_texts, padding=True, truncation=True, return_tensors="pt")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Prediksi Sampel:


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
encoded = {k: v.to(device) for k, v in encoded.items()}

In [ ]:
with torch.no_grad():
    outputs = model(**encoded)
    logits = outputs.logits
    probs = torch.nn.functional.softmax(logits, dim=-1)
    preds = torch.argmax(probs, dim=-1)

In [ ]:
for text, pred, prob in zip(sample_texts, preds, probs):
    label = le.inverse_transform([pred.item()])[0]
    confidence = prob[pred.item()].item()
    print(f"Text: {text}")
    print(f"Predicted Sentiment: {label} (Confidence: {confidence:.4f})")
    print("-" * 50)

Text: Akun saya tidak bisa login, sangat mengecewakan
Predicted Sentiment: negative (Confidence: 0.7764)
--------------------------------------------------
Text: Update terbaru sangat membantu, makin keren
Predicted Sentiment: positive (Confidence: 0.6493)
--------------------------------------------------
Text: Biasa saja, belum terlalu terasa bedanya
Predicted Sentiment: negative (Confidence: 0.4321)
--------------------------------------------------
